The below script has no GUI interface but uses OPENAI model along with Chunks and Delay to reduced TTS API Error 

In [1]:
import PyPDF2
import pyttsx3
import tkinter as tk
from tkinter import *
from tkinter import Label, Button, filedialog, ttk
from tkinter import messagebox as msgb
import subprocess
from openai import OpenAI
from gtts import gTTS
import os
import shutil
import time
import re

In [2]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = []
            for page in pdf_reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text.append(page_text.strip())
            return " ".join(text)
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

# Function to summarize text using OpenAI API (new method)
def summarize_text_with_openai(text):
    try:
        openai.api_key = os.getenv("OPENAI_API_KEY")  # Make sure your OpenAI key is set here
        
        # Using 'gpt-3.5-turbo' model, which works well for conversational and summarization tasks
        # Set a reasonable chunk size to avoid token limits
        chunk_size = 1500  # Tokens limit for GPT-3.5 is around 4096 tokens, so 1500 characters are safe
        summarized_text = []
        
        # Split the text into chunks if it exceeds the chunk size (for long PDFs)
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i + chunk_size]
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"Summarize the following text for an audiobook:\n{chunk}"}
                ],
                max_tokens=150,
                temperature=0.5
            )
            summarized_text.append(response['choices'][0]['message']['content'].strip())

        # Combine all summarized chunks
        return " ".join(summarized_text)
    except Exception as e:
        print(f"Error using OpenAI API: {e}")
        return text  # Return the original text in case of error

# Function to convert text to speech using gTTS
def text_to_speech(text, output_audio_path):
    max_retries = 3
    retry_delay = 5  # seconds
    for attempt in range(max_retries):
        try:
            tts = gTTS(text, lang='en')
            tts.save(output_audio_path)
            print(f"Audio file saved as {output_audio_path}")
            break
        except Exception as e:
            if '429' in str(e):
                print(f"Rate limit error: {e}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print(f"Error in text-to-speech conversion: {e}")
                break

# Main function
def pdf_to_speech(pdf_path, output_audio_path):
    print("Extracting text from PDF...")
    text = extract_text_from_pdf(pdf_path)
    if text:
        print("Summarizing text with OpenAI API...")
        summarized_text = summarize_text_with_openai(text)
        print("Converting text to speech...")
        text_to_speech(summarized_text, output_audio_path)
    else:
        print("Failed to extract text from PDF.")

# Usage example
if __name__ == "__main__":
    pdf_path = ""  # Replace with your PDF file path
    output_audio_path = "EQDC_NISM.mp3"  # Output audio file path
    pdf_to_speech(pdf_path, output_audio_path)


Extracting text from PDF...
Error reading PDF: [Errno 2] No such file or directory: ''
Failed to extract text from PDF.
